In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split, evaluate_keras

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate, train_test_split
from tensorflow import keras
from nltk.tokenize import TweetTokenizer

2022-06-04 01:40:05.782460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-04 01:40:05.782485: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize, max_features=10000), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize, max_features=10000), 
                CountVectorizer(ngram_range=(1,5), analyzer="char", max_features=10000), 
                CountVectorizer(ngram_range=(4,5), analyzer="char", max_features=10000), 
                CountVectorizer(ngram_range=(3,8), analyzer="char", max_features=10000), 
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize, max_features=10000),  
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize, max_features=10000), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char", max_features=10000), 
                TfidfVectorizer(ngram_range=(4,5), analyzer="char", max_features=10000), 
                TfidfVectorizer(ngram_range=(3,8), analyzer="char", max_features=10000)]

In [4]:
def build_model(input_shape, output_shape):  
    model = keras.models.Sequential(name="NeuralNetwork")
    model.add(keras.layers.Input(shape=input_shape, name="Input"))
    model.add(keras.layers.Dense(30, activation="relu", name="Dense1"))
    model.add(keras.layers.Dense(30, activation="relu", name="Dense2"))
    model.add(keras.layers.Dense(30, activation="relu", name="Dense3"))
    model.add(keras.layers.Dense(output_shape, activation="softmax", name="Output"))
    return model

In [5]:
evaluation = list()
usernames = list(np.unique(data["username"]))
results = list()

for i, vectorizer in enumerate(vectorizers):
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(
                data, author1, author2)

            scaler = MinMaxScaler()
            X_train = vectorizer.fit_transform(X_train["comment"]).toarray()
            X_test = vectorizer.transform(X_test["comment"]).toarray()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            y_classes = pd.get_dummies(y_train).columns
            y_train = pd.get_dummies(y_train).values
            y_test = pd.get_dummies(y_test).values
            
            input_shape_text = X_train.shape[1]
            output_shape = y_train.shape[1]

            callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

            model = build_model(input_shape_text, output_shape)
            model.compile(loss = "categorical_crossentropy",
                        optimizer = keras.optimizers.SGD(learning_rate=0.01),
                        metrics = ["accuracy"])
            history = model.fit(X_train, y_train, epochs=1000, callbacks=[callback], validation_split=0.1, shuffle=True, verbose=True)


            y_pred_proba = model.predict(X_test)
            evaluation.append(evaluate_keras(y_test.argmax(1), y_pred_proba, *y_classes))
    metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
    metrics["vectorizer"] = [vectorizer_str for i in range(len(metrics))]
    results.append(metrics)

Running => CountVectorizer(max_features=10000,
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f688e5ba850>>)


2022-06-04 01:40:09.028691: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-04 01:40:09.028725: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-04 01:40:09.028751: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lenovo-214): /proc/driver/nvidia/version does not exist
2022-06-04 01:40:09.029019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/1000
42/42 [==============================] - 1s 18ms/step - loss: 0.6933 - accuracy: 0.4890 - val_loss: 0.6923 - val_accuracy: 0.5306
Epoch 2/1000
42/42 [==============================] - 0s 6ms/step - loss: 0.6926 - accuracy: 0.5246 - val_loss: 0.6919 - val_accuracy: 0.5578
Epoch 3/1000
42/42 [==============================] - 0s 7ms/step - loss: 0.6919 - accuracy: 0.5465 - val_loss: 0.6912 - val_accuracy: 0.5306
Epoch 4/1000
42/42 [==============================] - 0s 6ms/step - loss: 0.6909 - accuracy: 0.5495 - val_loss: 0.6906 - val_accuracy: 0.5442
Epoch 5/1000
42/42 [==============================] - 0s 6ms/step - loss: 0.6898 - accuracy: 0.5261 - val_loss: 0.6901 - val_accuracy: 0.5918
Epoch 6/1000
42/42 [==============================] - 0s 6ms/step - loss: 0.6884 - accuracy: 0.6228 - val_loss: 0.6889 - val_accuracy: 0.5782
Epoch 7/1000
42/42 [==============================] - 0s 6ms/step - loss: 0.6866 - accuracy: 0.5865 - val_loss: 0.6876 - val_accuracy: 0.6531
Epoch

In [21]:
metrics_df = pd.DataFrame([results[i][:-1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][-1] for i in range(len(results))]
metrics_df.to_csv("../../results/neural_network.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.875800,0.875600,0.877600,0.876000,0.928600,"[CountVectorizer(max_features=10000,\n ..."
1,0.851400,0.851450,0.853750,0.851650,0.916100,"[CountVectorizer(max_features=10000, ngram_ran..."
2,0.841600,0.842467,0.844000,0.841767,0.911567,"[CountVectorizer(analyzer='char', max_features..."
3,0.830075,0.830825,0.832025,0.830200,0.904425,"[CountVectorizer(analyzer='char', max_features..."
4,0.825340,0.826420,0.827960,0.825500,0.900920,"[CountVectorizer(analyzer='char', max_features..."
5,0.828667,0.829583,0.830983,0.828817,0.906833,"[TfidfVectorizer(max_features=10000,\n ..."
6,0.831557,0.832271,0.833914,0.831771,0.911429,"[TfidfVectorizer(max_features=10000, ngram_ran..."
7,0.830537,0.831163,0.832600,0.830725,0.911237,"[TfidfVectorizer(analyzer='char', max_features..."
8,0.831422,0.831967,0.833367,0.831622,0.912278,"[TfidfVectorizer(analyzer='char', max_features..."
9,0.838520,0.839030,0.840290,0.838700,0.916610,"[TfidfVectorizer(analyzer='char', max_features..."
